In [3]:
data_dir="H:\\kaggle\\home_loans\\src"

In [4]:
import os
import csv
import re
from collections import namedtuple, defaultdict

In [44]:
test_fn = os.path.join(data_dir, "application_test.csv")
train_fn = os.path.join(data_dir, "application_train.csv")
bureau_fn = os.path.join(data_dir, "bureau.csv")
bureau_balance_fn = os.path.join(data_dir, "bureau_balance.csv")
card_balance_fn = os.path.join(data_dir, "credit_card_balance.csv")
instalment_payment_fn = os.path.join(data_dir, "installments_payments.csv")
prev_app_fn = os.path.join(data_dir, "previous_application.csv")
cash_balance_fn = os.path.join(data_dir, "POS_CASH_balance.csv")

In [45]:
def get_values_non_numeric(fn, col_name):
    bad_vals = set()
    with open(fn, "r") as fp:
        header = fp.readline().rstrip().split(",")
        cpos = dict([c, i] for i, c in enumerate(header))
        rdr = csv.reader(fp, delimiter=',', quotechar='"')
        for toks in rdr:
            val = toks[cpos[col_name]]
            try:
                float(val)
            except:
                bad_vals.add(val)
    return bad_vals

FieldStats = namedtuple("FieldStats", ("dtype", "min_len", "max_len", "precision", "has_na", "levels", "max_value", "min_value"))

def collect_field_stats(fns):
    ptrn = re.compile("^[+|-]?(?P<dec>[0-9]+)(\.(?P<frac>[0-9]*))?(e[+|-]?(?P<exp>[0-9]+))?$")
    has_na = defaultdict(lambda: False)
    min_len = defaultdict(lambda: float("inf"))
    max_len = defaultdict(lambda: 0)
    max_value = defaultdict(lambda: 0)
    min_value = defaultdict(lambda: float("inf"))
    precision = defaultdict(lambda: -1)
    dtypes = defaultdict(lambda: "int")
    levels = defaultdict(lambda: set())
    ret = {}
    
    
    for fn in fns:
        with open(fn, "r") as fp:
            rdr = csv.reader(fp, delimiter=',', quotechar='"')
            cols = list(next(rdr))
            cpos = dict([c, i] for i, c in enumerate(cols))
            for toks in rdr:
                assert(len(toks) == len(cols))
                for i, col in enumerate(cols):
                    val = toks[i]
                    lenval = len(val)
                    if lenval < min_len[col]:
                        min_len[col] = lenval
                    if lenval > max_len[col]:
                        max_len[col] = lenval
                    if lenval == 0:
                        has_na[col] = True
                    else:
                        try:
                            int(val)
                            if int(val) < min_value[col]:
                                min_value[col] = int(val)
                            if int(val) > max_value[col]:
                                max_value[col] = int(val)
                        except:
                            if dtypes[col] == "int":
                                dtypes[col] = "float"
                            try:
                                float(val)
                                m = ptrn.match(val)
                                dec = m.group("dec") or ""
                                frac = m.group("frac") or ""
                                exponent = m.group("exp")
                                if exponent:
                                    assert(int(exponent) > 0)
                                    exp_num = int(exponent)-1
                                else:
                                    exp_num = 0
                                prec = len(dec) + len(frac) + exp_num
                                if prec > precision[col]:
                                    precision[col] = prec
                                if float(val) < min_value[col]:
                                    min_value[col] = float(val)
                                if float(val) > max_value[col]:
                                    max_value[col] = float(val)
                            except Exception as e:
                                if col == "EXT_SOURCE_2":
                                    print(e, val, max_value[col], min_value[col])
                                dtypes[col] = "str"
                        levels[col].add(val)

    for k in min_len.keys():
        if dtypes[k] == "str":
            vals = levels[k]
        else:
            vals = []
        ret[k] = FieldStats(dtype=dtypes[k],
                            min_len=min_len[k],
                            max_len=max_len[k],
                            precision=precision[k],
                            has_na=has_na[k],
                            levels=vals,
                            max_value=max_value[k],
                            min_value=min_value[k])
    return ret
            
def get_col_names(fn):
    with open(fn, "r") as fp:
        rdr = csv.reader(fp, delimiter=',', quotechar='"')
        return next(rdr)

In [46]:
get_values_non_numeric(test_fn, "EXT_SOURCE_2")

{''}

In [64]:
stats = collect_field_stats([cash_balance_fn])

In [65]:
cols = get_col_names(cash_balance_fn)

for col in cols:
    if stats[col].dtype == "str":
        print(col.lower(), "=", "Column(String(%s))" % stats[col].max_len)
    elif stats[col].dtype == "float":
        print(col.lower(), "=", "Column(Float(precision=%s, asdecimal=True))" % stats[col].precision)
    elif stats[col].dtype == "int":
        print(col.lower(), "=", "Column(Integer)")
    else:
        raise ValueError("Unknown type %s", stats[col.dtype])
    
for col in cols:
    if "FLAG" in col:
        print('"%s": partial(convert_bool, true_value="1", false_value="0", missing_symbols=MISSING_VALS),' % col.lower())

sk_id_prev = Column(Integer)
sk_id_curr = Column(Integer)
months_balance = Column(Integer)
cnt_instalment = Column(Float(precision=3, asdecimal=True))
cnt_instalment_future = Column(Float(precision=3, asdecimal=True))
name_contract_status = Column(String(21))
sk_dpd = Column(Integer)
sk_dpd_def = Column(Integer)


In [57]:
stats["STATUS"].levels

{'0', '1', '2', '3', '4', '5', 'C', 'X'}

In [11]:
get_values_non_numeric

<function __main__.get_values_non_numeric(fn, col_name)>

In [45]:
for k, v in stats.items():
    if v.dtype == "str":
        print(k, list(v.levels)[:50])

NAME_TYPE_SUITE ['Group of people', 'Unaccompanied', 'Spouse, partner', 'Other_A', 'Children', 'Family', 'Other_B']
EMERGENCYSTATE_MODE ['Yes', 'No']
NAME_EDUCATION_TYPE ['Lower secondary', 'Academic degree', 'Incomplete higher', 'Secondary / secondary special', 'Higher education']
FLAG_OWN_REALTY ['N', 'Y']
FLAG_OWN_CAR ['N', 'Y']
NAME_INCOME_TYPE ['Commercial associate', 'Pensioner', 'Student', 'Unemployed', 'Businessman', 'State servant', 'Maternity leave', 'Working']
NAME_HOUSING_TYPE ['House / apartment', 'Office apartment', 'Co-op apartment', 'Municipal apartment', 'Rented apartment', 'With parents']
NAME_CONTRACT_TYPE ['Revolving loans', 'Cash loans']
FONDKAPREMONT_MODE ['reg oper spec account', 'not specified', 'reg oper account', 'org spec account']
CODE_GENDER ['XNA', 'F', 'M']
ORGANIZATION_TYPE ['Construction', 'Trade: type 3', 'Industry: type 3', 'Industry: type 2', 'Industry: type 5', 'Business Entity Type 3', 'Industry: type 4', 'Realtor', 'Mobile', 'Trade: type 7', 'Tran

In [16]:
from decimal import Decimal



In [17]:
def test_decimal_float_difference(fn, colname, max_diff=10e-15):
    cols = get_col_names(fn)
    col_pos = cols.index(colname)
    with open(fn, "r") as fp:
        rdr = csv.reader(fp, quotechar='"', delimiter=',')
        next(rdr)  # skip header
        for line in rdr:
            if abs(Decimal(line[col_pos]) - Decimal(str(float(line[col_pos])))) > max_diff:
                return False
    return True

def test_int_float_equiv(fn, colname, max_diff=10e-7):
    cols = get_col_names(fn)
    col_pos = cols.index(colname)
    with open(fn, "r") as fp:
        rdr = csv.reader(fp, quotechar='"', delimiter=',')
        next(rdr)  # skip header
        for line in rdr:
            if not line[col_pos]:
                continue
            if abs(Decimal(int(float(line[col_pos]))) - Decimal(line[col_pos])) > max_diff:
                return False
    return True

In [63]:
test_cols = ["days_first_drawing", "days_first_due", "days_last_due_1st_version", "days_last_due", "days_termination", "nflag_insured_on_approval", "nflag_last_appl_in_day"]
#test_cols = ['cnt_drawings_atm_current']
for col in test_cols:
    print(col, test_int_float_equiv(prev_app_fn, col.upper()))

days_first_drawing True
days_first_due True
days_last_due_1st_version True
days_last_due True
days_termination True
nflag_insured_on_approval True
nflag_last_appl_in_day True


In [2]:
1=2

SyntaxError: can't assign to literal (<ipython-input-2-a0aef98f5518>, line 1)

In [ ]:
burea_stats = 